# VQE Tutorial (Part 2)
In this tutorial, we will run VQE experiments for $H_2$ molecule on both noisy simulators and quantum 
computers.

If you have any difficulties/questions/concerns, shoot an email to eag190@scarletmail.rutgers.edu

### Contents
3. Multiple Distances, Noisy Simulator
4. Multiple Distances, Quantum Computer

*By distance, I am referring to the interatomic separation of the 2 hydrogen atoms.*


### 0.1 Some Imports

In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [ ]:
from qiskit.aqua.algorithms import VQE, ExactEigensolver
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.providers.aer import noise
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.aqua.operators import Z2Symmetries
from qiskit.providers.aer.noise import NoiseModel

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### 0.2 Switch to Parity Basis

Parity Basis will involve a 2 qubit reduction, which will really simplify things. Also the more gates and qubits in your circuit, the more erroneous your result. So you really want simple here. 

Now what is parity basis. Lets start with occupation numbers. Whether a spin orbital is occupied or not depends on the occupation number $o$. If $o = 1$, then the orbital is occupied, and if $o = 0$, then the oribital is unoccupied. 

In Jordan Wigner, $q_i = o_i$ i.e. the occupation number of an orbital is equivalent to the state of the corresponding qubit. So if the $q_i = |1>$ then the corresponding spin orbital has an occupation number $o_i = 1$.

In Parity Basis , $ q_i = o_0 + o_1 +o_2 + ... + o_i \mod 2$. So if $o_0 = 1, o_1 = 0, o_2 = 1$, then
$$q_0 = o_0 \mod 2 = 1 \mod 2 = 1$$
$$q_1 = o_0 + o_1 \mod 2 = 1 +0 \mod 2 = 1$$
$$q_2 = o_0 + o_1 + o_2 \mod 2 = 1 + 0 + 1 \mod 2 = 0$$

It's just another way of encoding electrons and spin orbitals onto quantum computers

*$a \mod b$ is the remainder when $a$ is divided by $b$*

In [ ]:
def get_qubit_ops(dist): 
    # Defining Molecule
    mol = 'H .0 .0 .0 ; H .0 .0 {}'
    driver = PySCFDriver(mol.format(dist), unit=UnitsType.ANGSTROM,
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    # Mapping to Qubit Hamiltonian
    map_type = 'parity'

    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    nuclear_repulsion_energy = molecule.nuclear_repulsion_energy
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    hf_energy = molecule.hf_energy 
    print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
    print("# of electrons: {}".format(num_particles))
    print("# of spin orbitals: {}".format(num_spin_orbitals))
    
    # and if PARITY mapping is selected, reduction qubits
    qubit_reduction = True if map_type == 'parity' else False

    ferOp = FermionicOperator(h1=h1, h2=h2)
    qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) if qubit_reduction else qubitOp
    qubitOp.chop(10**-10)
    shift = repulsion_energy
    
    return qubitOp, num_particles, num_spin_orbitals, shift, hf_energy

## 3. Multiple Distances, Noisy Simulator

Here we will try to act as if we are running on real quantum computer when in reality, we aren't. 

Noise Models are very important if your experiment involves lots of circuit evaluations which may or may not be practical on an real device. If you can simulate the conditions of the real device however, you may can run those circuits and get somewhat approximate answer in a lot less time.

### 3.1 Constructing Noise Model 

Here we will mimick the noise of some quantum computer in our simulation. 

In [ ]:
#choose a machine...you can do london, ourense (not sure about spelling there!), essex, burlington 
## ... any quantum computer with at least 2 qubits. For the yorktown one, inside the code 
## ...  machine = provider.get_backend( *** ), write *** = 'ibmqx2'

machine_str = 'burlington'

#noise model 
provider = IBMQ.get_provider(hub='ibm-q')
machine = provider.get_backend('ibmq_'+machine_str)
noise_model = NoiseModel.from_backend(machine)
#Ideal simulator 
simulator = Aer.get_backend('qasm_simulator')
#noisy simulator
noisy_sim = QuantumInstance(backend = simulator, noise_model = noise_model)

### 3.2 Running on the Noise Model

In [ ]:
# I am going to do less distances, because it takes a lot of time
distances = np.arange(0.4, 1.0, 0.1)
exact_energies = []
vqe_noisy_energies = []
vqe_ideal_energies = []

#optimizer
optimizer = COBYLA(maxiter=100)

for dist in distances:
    print('------------Interatomic Distance = '+ str(np.round(dist, 2)) + '--------------')
    qubitOp, num_particles, num_spin_orbitals, shift, hf_energy = get_qubit_ops(dist)
    # Finding exact energies to show how off VQE was from exact energy curve
    result = ExactEigensolver(qubitOp).run()
    exact_energies.append(result['energy'] + shift)
    # Initial STate 
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        'parity'
    ) 
    
    # UCCSD Variational Form
    var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    # Running VQE using a ideal simulator 
    vqe = VQE(qubitOp, var_form, optimizer, include_custom = True)
    result = vqe.run(simulator)['energy'] + shift
    vqe_ideal_energies.append(result)
    
    # Running VQE using a noisy simulator
    vqe = VQE(qubitOp, var_form, optimizer, include_custom = True)
    resulty = vqe.run(noisy_sim)['energy'] + shift
    vqe_noisy_energies.append(resulty)
    print("Ideal VQE Result:", vqe_ideal_energies[-1], "Noisy VQE Result:", vqe_noisy_energies[-1], "Exact Energy:", 
          exact_energies[-1])
print("All energies have been calculated")

### 3.3 Plotting the Results

In [ ]:
# plotting the data
plt.plot(distances, exact_energies, label="Exact Energy")
plt.plot(distances, vqe_ideal_energies, label="Ideal VQE Energy")
plt.plot(distances, vqe_noisy_energies, label="Noisy VQE Energy")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy (Hartree)')
plt.title('Dissassociation Curves for Hydrogen Molecule')
plt.legend()
plt.show()

ooooh let's look zoom in on difference from exact energy

In [ ]:
vqe_ideal_difference= []
vqe_noisy_difference=[]
for i in range(len(vqe_ideal_energies)):
    vqe_ideal_difference = vqe_ideal_difference + [vqe_ideal_energies[i] - exact_energies[i]]
    vqe_noisy_difference = vqe_noisy_difference + [vqe_noisy_energies[i] - exact_energies[i]]

plt.plot(distances, vqe_ideal_difference, color= 'tab:orange', label="Ideal VQE")
plt.plot(distances, vqe_noisy_difference, color='tab:green', label="Noisy VQE")
plt.title('Difference from Exact Energy')
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy (Hartree)')
plt.legend()
plt.show()

Here you can see that noisy simulator is really really bad! Remember chemical accuracy is $10^{-3}$ Hartree. So the simulated noise is making a difference.

#  4. Multiple Distances, Quantum Computer

### 4.1 Specifying the Device

Ok the way the code is structured, its gonna use the same machine as the one you specified above for noise model. If you want to change around some things, just delete the '#' symbol and edit the string inside the parentheses

In [ ]:
#machine = provider.get_backend('ibmq_burlington')

### 4.2 Running on the Device
This will take a lot of time (40 min - 2 hrs) so beware! But the results are worth it so ayeeee

In [ ]:
#storage tank
vqe_qc_energies = []

#begin VQE
for dist in distances:
    print('------------Interatomic Distance = '+ str(np.round(dist, 2)) + '--------------')
    qubitOp, num_particles, num_spin_orbitals, shift, hf_energy = get_qubit_ops(dist)
    # Initial STate 
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        'parity'
    ) 
    #print(qubitOp.num_qubits)
    # UCCSD Variational Form
    var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    
    # Running VQE using a simulator, not actual quantum computer
    vqe = VQE(qubitOp, var_form, optimizer, include_custom = True)
    resulty = vqe.run(machine)['energy'] + shift
    vqe_noisy_energies.append(resulty)
    print("VQE-on-QC Energy:", vqe_qc_energies[-1], "Exact Energy:", 
          exact_energies[-1])
print("All energies have been calculated")

### 4.3 Plotting the Results

In [ ]:
# plotting the data
plt.plot(distances, exact_energies, label="Exact Energy")
plt.plot(distances, vqe_ideal_energies, label="Ideal VQE Energy")
plt.plot(distances, vqe_noisy_energies, label="Noisy VQE Energy")
plt.plot(distances, vqe_qc_energies, label="QC VQE Energy")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy (Hartree)')
plt.title('Dissassociation Curves for Hydrogen Molecule')
plt.legend()
plt.show()

If your noise model belongs to same machine as your QC-VQE's quantum computer, you can already see how well (not exact, but well enough) the noise model mimicks the actual device.

In [ ]:
vqe_qc_difference= []
for i in range(len(vqe_qc_energies)):
    vqe_qc_difference = vqe_qc_difference + [vqe_qc_energies[i] - exact_energies[i]]

plt.plot(distances, vqe_ideal_difference, color= 'tab:orange', label="Ideal VQE")
plt.plot(distances, vqe_noisy_difference, color='tab:green', label="Noisy VQE")
plt.plot(distances, vqe_qc_difference, color='tab:red', label="QC VQE")
plt.title('Difference from Exact Energy')
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy (Hartree)')
plt.legend()
plt.show()

Again VQE on quantum computers of today is not that great! That's why there is research being done in error correction and error mitigation. 

If you enjoyed doing this and want to do more molecules/find out more info, refer to the links below: 

https://qiskit.org/textbook/ch-applications/vqe-molecules.html

https://github.com/qiskit-community/qiskit-community-tutorials/tree/master/chemistry

# Thank You!
With that , peep peep! Hope you learned something about QC's from all this. Happy Senior year!

-- Eesh